In [ ]:
%pip install -r requirements.txt

import os, re
from train import TrainingRunArguments, do_training_run

def get_next_run_name(model):
    pattern = re.compile(model + r"-rev(\d+)$")
    max_rev = 0

    for folder in os.listdir("models/"):
        match = pattern.search(folder)
        if match:
            max_rev = max(max_rev, int(match.group(1)))

    return f"{model}-rev{max_rev + 1}"

os.environ["HF_HOME"] = "/workspace/"

## Generate Data

In [ ]:
%pip install -r data/requirements.txt
from data.generate_home_assistant_data import main as generate_data

generate_data(["--train", "--test", "--large", "--sharegpt", "--language", "english", "german", "french", "spanish", "polish"])

## Llama 3.2 1B

In [ ]:
# python3 train.py \
#     --run_name Home-Llama-3.2-1B-rev1 \
#     --base_model meta-llama/Llama-3.2-1B-Instruct \
#     --bf16 \
#     --train_dataset data/home_assistant_train.jsonl \
#     --test_dataset data/home_assistant_test.jsonl \
#     --learning_rate 2e-5 --learning_rate_warmup 0.03 --batch_size 64 --epochs 1 \
#     --micro_batch_size 2 \
#     --ctx_size 2048 \
#     --save_steps 200 --save_total_limit 1 --eval_steps 200 --logging_steps 2

do_training_run(TrainingRunArguments(
    run_name=get_next_run_name("Home-Llama-3.2-1B"),
    base_model="meta-llama/Llama-3.2-1B-Instruct",
    bf16=True,
    train_dataset="data/home_assistant_train.jsonl",
    test_dataset="data/home_assistant_test.jsonl",
    learning_rate=2e-5, learning_rate_warmup=0.03, 
    batch_size=64, micro_batch_size=2, epochs=1,
    ctx_size=2048,
    save_steps=200, save_total_limit=1, eval_steps=200, logging_steps=2,
))

# Qwen3

In [ ]:
do_training_run(TrainingRunArguments(
    run_name=get_next_run_name("Home-Qwen-3-1.7B"),
    base_model="Qwen/Qwen3-1.7B",
    bf16=True,
    train_dataset="data/home_assistant_train.jsonl",
    test_dataset="data/home_assistant_test.jsonl",
    learning_rate=2e-5, learning_rate_warmup=0.03, 
    batch_size=64, micro_batch_size=2, epochs=1,
    ctx_size=2048,
    save_steps=200, save_total_limit=1, eval_steps=200, logging_steps=2,
))